# ValidMind for model validation 4 — Finalize testing and reporting

## Prerequisites

## Setting up

## Adjust a ValidMind test

## Run diagnostic tests

## Run feature importance tests

## Implement a custom test

## Verify test runs

## In summary

## Next steps

### Work with your validation report